In [1]:
import os

os.listdir("/kaggle/input")

['dogs-vs-cats', 'dogs-vs-cats-redux-kernels-edition', 'kaggle-json']

In [2]:
for folder in ['kaggle-json', 'dogs-vs-cats', 'dogs-vs-cats-redux-kernels-edition']:
    print(f"\nContents of: /kaggle/input/{folder}")
    print(os.listdir(f"/kaggle/input/{folder}")[:5])


Contents of: /kaggle/input/kaggle-json
['kaggle.json']

Contents of: /kaggle/input/dogs-vs-cats
['sample_submission.csv', 'test', 'train']

Contents of: /kaggle/input/dogs-vs-cats-redux-kernels-edition
['sample_submission.csv', 'train.zip', 'test.zip']


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, EfficientNetB0, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
import matplotlib.pyplot as plt
import time

2025-04-23 01:52:17.768242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745373138.021054     998 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745373138.090147     998 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Paths
DATA_DIR = "/kaggle/working/split_dataset"

In [ ]:
import os

data_dir = "/kaggle/input/dogs-vs-cats/train/train"
print("Sample files:", os.listdir(data_dir)[:5])

In [ ]:
import os, shutil

data_dir = "/kaggle/input/dogs-vs-cats/train/train"
split_dir = "/kaggle/working/split_dataset"
cat_dir = os.path.join(split_dir, "cats")
dog_dir = os.path.join(split_dir, "dogs")

os.makedirs(cat_dir, exist_ok=True)
os.makedirs(dog_dir, exist_ok=True)

for fname in os.listdir(data_dir):
    src = os.path.join(data_dir, fname)
    if fname.startswith("cat"):
        shutil.copy(src, os.path.join(cat_dir, fname))
    elif fname.startswith("dog"):
        shutil.copy(src, os.path.join(dog_dir, fname))

In [ ]:
print("Cats:", len(os.listdir(cat_dir)))
print("Dogs:", len(os.listdir(dog_dir)))


In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

# Load raw datasets
raw_train_ds = image_dataset_from_directory(
    split_dir,
    labels="inferred",
    label_mode="binary",
    batch_size=16,
    image_size=(224, 224),
    validation_split=0.2,
    subset="training",
    seed=42
)

raw_val_ds = image_dataset_from_directory(
    split_dir,
    labels="inferred",
    label_mode="binary",
    batch_size=16,
    image_size=(224, 224),
    validation_split=0.2,
    subset="validation",
    seed=42
)


In [ ]:
# Store class names for later use
class_names = raw_train_ds.class_names

In [ ]:
from tensorflow.keras import layers

# Define data augmentation (only for training)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

# Define normalization
normalization_layer = layers.Rescaling(1./255)
AUTOTUNE = tf.data.AUTOTUNE

# Apply data augmentation + normalization to training set
train_ds = raw_train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

# Apply only normalization to validation set
val_ds = raw_val_ds.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# for images, labels in train_ds.take(1):
#     print(images.shape, labels.shape)

In [ ]:
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         img = tf.keras.utils.array_to_img(images[i])  # handles float32 to uint8 safely
#         label = "Dog" if labels[i].numpy() == 1 else "Cat"
#         plt.imshow(img)
#         plt.title(label)
#         plt.axis("off")


In [ ]:
# from collections import Counter

# counter = Counter()
# for _, label in train_ds.unbatch():
#     counter[int(label.numpy())] += 1

# print("Cat count:", counter[0])
# print("Dog count:", counter[1])

In [ ]:
# for images, labels in train_ds.take(1):
#     print("Image batch shape:", images.shape)  # Should be (batch_size, 224, 224, 3)
#     print("Label batch shape:", labels.shape)

In [ ]:
# # Run this twice and compare outputs manually
# for images, _ in train_ds.take(1):
#     plt.imshow(tf.keras.utils.array_to_img(images[0]))
#     break

## MODEL EXPLORATION

### MOBILENET_V2

In [ ]:
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras import layers, models
# from tensorflow.keras.optimizers import Adam

# # Load MobileNetV2 base
# base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
# base_model.trainable = False  # freeze pretrained layers

# # Build full model
# model = models.Sequential([
#     base_model,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(1, activation='sigmoid')  # Binary classification
# ])

# # Compile
# model.compile(optimizer=Adam(learning_rate=0.001),
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

# model.summary()

In [ ]:
# # Train the model
# history_mobilenet = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=5  # Start with 5; increase if training looks good
# )

In [ ]:
# import matplotlib.pyplot as plt

# acc = history_mobilenet.history['accuracy']
# val_acc = history_mobilenet.history['val_accuracy']
# loss = history_mobilenet.history['loss']
# val_loss = history_mobilenet.history['val_loss']
# epochs_range = range(len(acc))

# plt.figure(figsize=(12, 4))

# plt.subplot(1, 2, 1)
# plt.plot(epochs_range, acc, label='Train Accuracy')
# plt.plot(epochs_range, val_acc, label='Val Accuracy')
# plt.legend()
# plt.title("MobileNetV2 - Accuracy")

# plt.subplot(1, 2, 2)
# plt.plot(epochs_range, loss, label='Train Loss')
# plt.plot(epochs_range, val_loss, label='Val Loss')
# plt.legend()
# plt.title("MobileNetV2 - Loss")

# plt.show()

### EFFICIENTNETB7

In [ ]:
# from tensorflow.keras.applications import EfficientNetB7
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import GlobalAveragePooling2D

# # Load pretrained EfficientNetB7 and extract features
# effnet = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# effnet.trainable = False

# feature_model = Model(inputs=effnet.input, outputs=GlobalAveragePooling2D()(effnet.output))

In [ ]:
# import numpy as np
# from tqdm import tqdm  # Optional progress bar

# def extract_features(dataset, model):
#     features = []
#     labels = []
#     for images, lbls in tqdm(dataset, desc="Extracting features"):
#         # Predict without printing progress
#         f = model.predict(images, verbose=0)
#         features.append(f)
#         labels.append(lbls.numpy())
#     return np.concatenate(features), np.concatenate(labels)

# # Run extraction
# X_train_feat, y_train = extract_features(train_ds, feature_model)
# X_val_feat, y_val = extract_features(val_ds, feature_model)

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score

# lr = LogisticRegression(max_iter=1000)
# lr.fit(X_train_feat, y_train)

# # Predict and evaluate
# y_pred_lr = lr.predict(X_val_feat)
# acc_lr = accuracy_score(y_val, y_pred_lr)
# print(f"✅ Logistic Regression Accuracy: {acc_lr:.4f}")

In [ ]:
# from xgboost import XGBClassifier

# xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
# xgb.fit(X_train_feat, y_train)

# # Predict and evaluate
# y_pred_xgb = xgb.predict(X_val_feat)
# acc_xgb = accuracy_score(y_val, y_pred_xgb)
# print(f"✅ XGBoost Accuracy: {acc_xgb:.4f}")

### SIMPLE CNN

In [ ]:
# from tensorflow.keras import layers, models

# simple_cnn = models.Sequential([
#     layers.Input(shape=(224, 224, 3)),
    
#     layers.Conv2D(32, (3, 3), activation='relu'),
#     layers.MaxPooling2D(),
    
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D(),
    
#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.MaxPooling2D(),
    
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dropout(0.3),
#     layers.Dense(1, activation='sigmoid')  # Binary classification
# ])

# simple_cnn.compile(optimizer='adam',
#                    loss='binary_crossentropy',
#                    metrics=['accuracy'])

# simple_cnn.summary()


In [ ]:
# history_cnn = simple_cnn.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=5
# )

In [ ]:
# from tensorflow.keras.applications import Xception
# from tensorflow.keras import layers, models
# from tensorflow.keras.optimizers import Adam

# # Load pretrained Xception
# xception_base = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# xception_base.trainable = False  # Freeze base layers

# # Build model
# xception_model = models.Sequential([
#     xception_base,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(1, activation='sigmoid')
# ])

# # Compile
# xception_model.compile(optimizer=Adam(learning_rate=0.001),
#                        loss='binary_crossentropy',
#                        metrics=['accuracy'])

# xception_model.summary()

In [ ]:
# history_xception = xception_model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=5
# )

### EFFICIENTNET B0

In [ ]:
# from tensorflow.keras.applications import EfficientNetB0

# efn_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# efn_base.trainable = False

# efn_model = models.Sequential([
#     efn_base,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(1, activation='sigmoid')
# ])

# efn_model.compile(optimizer=Adam(learning_rate=0.001),
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])

# efn_model.summary()


In [ ]:
# history_efn = efn_model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=5
# )

## TEST DATA PREPROCESSING

In [ ]:
import tensorflow as tf
import os
import numpy as np

# Define a function to preprocess test images (normalization only)
def preprocess_test_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = tf.cast(img, tf.float32) / 255.0
    return img

# Get all test image file paths
test_dir = "/kaggle/input/dogs-vs-cats/test/test"
test_images = sorted(
    [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith('.jpg')],
    key=lambda x: int(os.path.basename(x).split('.')[0])
)


# Create a dataset from file paths
test_ds = tf.data.Dataset.from_tensor_slices(test_images)
test_ds = test_ds.map(lambda x: preprocess_test_image(x), num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

# Now you can use this dataset for predictions
X_test = np.concatenate([batch.numpy() for batch in test_ds], axis=0)

In [ ]:
print("Min pixel value in test data:", X_test.min())
print("Max pixel value in test data:", X_test.max())

In [ ]:
print("Shape of one image:", X_test[0].shape)  # Should print (224, 224, 3)
print("Total number of test images:", X_test.shape[0])

BEST MODEL IS EFFIECIENTNET

## TESTING XCEPTION MODEL SINCE IT GAVE GOOD RESULTS

In [ ]:
# from tensorflow.keras import layers, models
# from tensorflow.keras.optimizers import Adam

# # Load pretrained Xception
# xception_base = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# xception_base.trainable = False  # Freeze base layers

# # Build model
# xception_model = models.Sequential([
#     xception_base,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(1, activation='sigmoid')
# ])

# # Compile
# xception_model.compile(optimizer=Adam(learning_rate=0.001),
#                        loss='binary_crossentropy',
#                        metrics=['accuracy'])

# xception_model.summary()

In [ ]:
# history_xception = xception_model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=5
# )

### J

#### USING MOBILENET V2 [TRANSFER LEARNING]

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 base
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # freeze pretrained layers

# Build full model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Train the model
history_mobilenet = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5  # Start with 5; increase if training looks good
)

In [ ]:
preds = model.predict(X_test, batch_size=16)  # or use 64 depending on memory

In [ ]:
import pandas as pd

ids = [int(fname.split('.')[0]) for fname in test_filenames]
submission = pd.DataFrame({
    "id": ids,
    "label": preds.flatten()  # Flatten to 1D array
})
submission.sort_values("id", inplace=True)
submission.to_csv("submission.csv", index=False)

### NOTE: I HAVE COMMENTEDD OUT THE MODEL EXPLORATION AND OTHER EDA PART FOR QUICK EXECUTION IN THE KAGGLE 